In [1]:
from langchain_core.prompts import PromptTemplate

common_prompt_template = PromptTemplate.from_template(
    """You are an expert in {dialect}. Your job is to read and understand the following [Database Schema] description, along with any [Reference Information], and then use your knowledge of {dialect} to generate an SQL statement that answers the [User Question]. Pay attention to the [Database Schema], only use tables and columns that are in the [Database Schema]. Avoid using any other tables or columns that are not in the [Database Schema].

[User Question]
{user_question}

[Database Schema]
{schema}

[Reference Information]
None

ONLY OUTPUT THE SQL STATEMENT, NO OTHER TEXT.
"""
)
xiyan_en_prompt_template = PromptTemplate.from_template(
    """You are an expert in {dialect}. Your job is to read and understand the following [Database Schema] description, along with any [Reference Information], and then use your knowledge of {dialect} to generate an SQL statement that answers the [User Question].

[User Question]
{user_question}

[Database Schema]
{schema}

[Reference Information]
None

[User Question]
{user_question}
```sql
"""
)
xiyan_cn_prompt_template = PromptTemplate.from_template(
    """你是一名{dialect}专家，现在需要阅读并理解下面的【数据库schema】描述，以及可能用到的【参考信息】，并运用{dialect}知识生成sql语句回答【用户问题】。
【用户问题】
{user_question}

【数据库schema】
{schema}

【参考信息】
None

【用户问题】
{user_question}

```sql"""
)

In [2]:
model_config = {
    "llama32_3b": {
        "model_name": "llama3.2:3b",
        "model_config_init": {
            "temperature": 0.0,
        },
        "prompt_template": common_prompt_template,
        "inference_type": "ollama",
    },
    "sqlllama_7b_16": {
        "model_name": "hf.co/mradermacher/SQL-Llama-v0.5-GGUF:F16",
        "model_config_init": {
            "temperature": 0.0,
        },
        "prompt_template": common_prompt_template,
        "inference_type": "ollama",
    },
    "xiyansql_7b_8_en_prompt": {
        "model_name": "hf.co/mradermacher/XiYanSQL-QwenCoder-7B-2504-GGUF:Q8_0",
        "model_config_init": {
            "temperature": 0.0,
        },
        "prompt_template": xiyan_en_prompt_template,
        "inference_type": "ollama",
    },
    "xiyansql_7b_8_cn_prompt": {
        "model_name": "hf.co/mradermacher/XiYanSQL-QwenCoder-7B-2504-GGUF:Q8_0",
        "model_config_init": {
            "temperature": 0.0,
        },
        "prompt_template": xiyan_cn_prompt_template,
        "inference_type": "ollama",
    },
    "xiyansql_7b_16_en_prompt": {
        "model_name": "hf.co/mradermacher/XiYanSQL-QwenCoder-7B-2504-GGUF:F16",
        "model_config_init": {"temperature": 0.1, "top_p": 0.8},
        "prompt_template": xiyan_en_prompt_template,
        "inference_type": "ollama",
    },
    "xiyansql_7b_16_cn_prompt": {
        "model_name": "hf.co/mradermacher/XiYanSQL-QwenCoder-7B-2504-GGUF:F16",
        "model_config_init": {"temperature": 0.1, "top_p": 0.8},
        "prompt_template": xiyan_cn_prompt_template,
        "inference_type": "ollama",
    },
    "xiyansql_7b_16_en_prompt_vllm": {
        "model_name": "XGenerationLab/XiYanSQL-QwenCoder-7B-2504",
        "model_config_init": {"temperature": 0.1, "top_p": 0.8},
        "prompt_template": xiyan_en_prompt_template,
        "inference_type": "vllm",
    },
}

In [3]:
from langchain_ollama import ChatOllama
from langchain_community.llms import VLLM
import pandas as pd
from tqdm.notebook import tqdm

tqdm.pandas()


class ModelCreater:
    def __init__(self):
        self.llm = None
        self.model_name = None
        self.infer_type = None

    def create_model(self, model_config: dict):
        if (
            self.llm
            and self.model_name == model_config["model_name"]
            and self.infer_type == model_config["inference_type"]
        ):
            return self.llm
        else:
            # shutdown created vllm
            if self.infer_type == "vllm":
                pass

            if model_config["inference_type"] == "ollama":
                self.llm = ChatOllama(
                    model=model_config["model_name"],
                    **model_config["model_config_init"],
                )
            elif model_config["inference_type"] == "vllm":
                self.llm = VLLM(
                    model=model_config["model_name"],
                    trust_remote_code=True,
                    **model_config["model_config_init"],
                )
            else:
                raise ValueError(
                    f"Inference type {model_config['inference_type']} not supported"
                )
        self.model_name = model_config["model_name"]
        self.infer_type = model_config["inference_type"]
        return self.llm


model_creater = ModelCreater()


def benchmark_model(
    model: str, input_file_path: str, output_file_path: str, dialect: str
):
    config = model_config[model]
    llm = model_creater.create_model(config)
    prompt_template = config["prompt_template"]

    def generate_sql(user_question: str, schema_list: list[str], dialect: str):
        message = prompt_template.invoke(
            {
                "user_question": user_question,
                "schema": "\n".join(schema_list),
                "dialect": dialect,
            }
        )
        message = llm.invoke(message)
        if not isinstance(message, str):
            message = message.content
        # format the message to be a valid sql statement
        message = (
            message.strip()
            .replace("```sql", "")
            .replace("```", "")
            .replace("\n", "")
            .replace("\t", "")
        )
        # print(f"Generated answer for question: {user_question[:20]}...")
        return message

    df = pd.read_csv(input_file_path)
    inference_df = df.assign(
        answer=lambda df_: df_.progress_apply(
            lambda row: generate_sql(row.question, row.schemas, dialect), axis=1
        )
    )
    with open(output_file_path, "w+") as f:
        answer_list = inference_df["answer"].tolist()
        f.write("\n".join(answer_list))

    return inference_df

In [4]:
test_list = [
    # "llama32_3b",
    # "sqlllama_7b_16",
    # "xiyansql_7b_8_en_prompt",
    # "xiyansql_7b_8_cn_prompt",
    # "xiyansql_7b_16_en_prompt",
    # "xiyansql_7b_16_cn_prompt",
    "xiyansql_7b_16_en_prompt_vllm",
]

In [5]:
for dataset in ["test"]:
    for model_name in test_list:
        dialect = "sqlite"
        input_file_path = f"prepare_data/{dataset}_input.csv"
        output_file_path = f"inference_data/{model_name}_{dataset}_inf.txt"
        print(f"Benchmarking {model_name}, {dataset} dataset")
        inference_df = benchmark_model(
            model_name, input_file_path, output_file_path, dialect
        )

Benchmarking xiyansql_7b_16_en_prompt_vllm, test dataset
INFO 06-23 09:50:10 [__init__.py:244] Automatically detected platform cuda.
INFO 06-23 09:50:35 [config.py:823] This model supports multiple tasks: {'embed', 'classify', 'score', 'reward', 'generate'}. Defaulting to 'generate'.
INFO 06-23 09:50:44 [config.py:2195] Chunked prefill is enabled with max_num_batched_tokens=8192.
INFO 06-23 09:50:47 [core.py:455] Waiting for init message from front-end.
INFO 06-23 09:50:47 [core.py:70] Initializing a V1 LLM engine (v0.9.1) with config: model='XGenerationLab/XiYanSQL-QwenCoder-7B-2504', speculative_config=None, tokenizer='XGenerationLab/XiYanSQL-QwenCoder-7B-2504', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=10240, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=Non

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 06-23 09:50:52 [default_loader.py:272] Loading weights took 3.36 seconds
INFO 06-23 09:50:53 [gpu_model_runner.py:1624] Model loading took 14.2488 GiB and 3.831651 seconds
INFO 06-23 09:51:01 [backends.py:462] Using cache directory: /home/user/.cache/vllm/torch_compile_cache/8895c6ff30/rank_0_0 for vLLM's torch.compile
INFO 06-23 09:51:01 [backends.py:472] Dynamo bytecode transform time: 7.74 s
INFO 06-23 09:51:07 [backends.py:135] Directly load the compiled graph(s) for shape None from the cache, took 5.224 s
INFO 06-23 09:51:08 [monitor.py:34] torch.compile takes 7.74 s in total
INFO 06-23 09:51:10 [gpu_worker.py:227] Available KV cache memory: 5.53 GiB
INFO 06-23 09:51:11 [kv_cache_utils.py:715] GPU KV cache size: 103,536 tokens
INFO 06-23 09:51:11 [kv_cache_utils.py:719] Maximum concurrency for 10,240 tokens per request: 10.11x
INFO 06-23 09:51:39 [gpu_model_runner.py:2048] Graph capturing finished in 29 secs, took 0.48 GiB
INFO 06-23 09:51:40 [core.py:171] init engine (profil

  0%|          | 0/2147 [00:00<?, ?it/s]

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

...

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]